In [1]:
import warnings
import numpy as np
import os
import pickle
import random

os.environ["CUDA_VISIBLE_DEVICES"] = '-1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.filterwarnings('ignore')

In [2]:
with open(r"./Data/dataset_n306_X_500_12.pickle", "rb") as input_file:
    dataset = pickle.load(input_file)

(x_dataset, y_dataset, labels) = dataset
permutation = list(range(x_dataset.shape[0]))

# if you need to shuffle the dataset
random.shuffle(permutation)

x_dataset = np.array([x_dataset[i] for i in permutation])
y_dataset = np.array([y_dataset[i] for i in permutation])
labels = np.array([labels[i] for i in permutation])

leads = [6, 7, 8] # V1 V2 V3
n_leads = len(leads)
x_dataset = np.take(x_dataset, leads, axis=-1)

In [ ]:
from search import *
from Models.model_buiders import *
from sklearn.model_selection import train_test_split, KFold
from accuracy_plotter import accuracy_plotter

n_out_fold = 5
kf_out = KFold(n_splits=n_out_fold)
results = []

for fold in range(n_out_fold):
    print("fold ", fold)
    train_val_index, test_index = list(kf_out.split(x_dataset))[fold]

    hp = {"units": 1200, "input_scaling": 1.0, "bias_scaling": 0.1, "inter_scaling": None, "spectral_radius": 0.999,
          "leaky": 0.1, "learning_rate": 0.01, "sub_reservoirs": n_leads}
    trainedIRESN = build_TrainableIRESN(hp)
    x_trainval, y_trainval,  = x_dataset[train_val_index], y_dataset[train_val_index],
    x_test, y_test = x_dataset[test_index], y_dataset[test_index]
    x_train, x_val, y_train, y_val = train_test_split(x_trainval, y_trainval, test_size=0.25)
    hist = trainedIRESN.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=1000,
                            batch_size=1000, verbose=1,
                            callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=100,
                                                                     restore_best_weights=True)])
    accuracy_plotter(hist, "./Plot/plot_" + str(fold))
    train_acc = trainedIRESN.evaluate(x_train, y_train, batch_size=1000)[1]
    val_acc = trainedIRESN.evaluate(x_val, y_val, batch_size=1000)[1]
    test_acc = trainedIRESN.evaluate_precise(x_test, y_test)

    result = {}
    result["train_acc"] = train_acc
    result["val_acc"] = val_acc
    result["test_acc"] = test_acc[0]
    result["sens"] = test_acc[1]
    result["spec"] = test_acc[2]
    result["hist"] = hist.history

    learned_hp = {"units": 1200, "input_scaling": list(trainedIRESN.trainable_weights[0].numpy().ravel()),
                  "bias_scaling": list(trainedIRESN.trainable_weights[1].numpy().ravel()),
                  # "inter_scaling": list(trainedIRESN.trainable_weights[2].numpy().ravel()),
                  "inter_scaling": None,
                  "spectral_radius": list(trainedIRESN.non_trainable_weights[0].numpy().ravel()),
                  "leaky": trainedIRESN.non_trainable_weights[1].numpy().ravel()[0], "sub_reservoirs": n_leads, "reg": None}
    result["learned_hp"] = learned_hp

    IRESNDirect = build_IRESN(learned_hp)
    IRESNDirect.reservoir = trainedIRESN.reservoir
    train_final, val_final, reg = k_fold_val_reg(IRESNDirect, x_trainval, y_trainval, n_fold=4)
    IRESNDirect.fit(x_trainval, y_trainval, reg=reg)
    test_final = IRESNDirect.evaluate_precise(x_test, y_test)
    readout_weights = IRESNDirect.readout.coef_
    result["weights"] = readout_weights

    result["reg"] = reg
    result["train_final"] = train_final
    result["val_final"] = val_final
    result["test_final"] = test_final[0]
    result["sens_final"] = test_final[1]
    result["spec_final"] = test_final[2]

    results.append(result)

In [8]:
train = []
val = []
test = []
spec = []
sens = []
train_f = []
val_f = []
test_f = []
sens_f = []
spec_f = []
hps = []
for mod in results:
    train.append(float(mod["train_acc"]))
    val.append(float(mod["val_acc"]))
    test.append(float(mod["test_acc"]))
    sens.append(float(mod["sens"]))
    spec.append(float(mod["spec"]))
    train_f.append(float(mod["train_final"]))
    val_f.append(float(mod["val_final"]))
    test_f.append(float(mod["test_final"]))
    sens_f.append(float(mod["sens_final"]))
    spec_f.append(float(mod["spec_final"]))
    hps.append(mod["learned_hp"])

train = np.array(train)
val = np.array(val)
test = np.array(test)
spec = np.array(spec)
sens = np.array(sens)
train_f = np.array(train_f)
val_f = np.array(val_f)
test_f = np.array(test_f)
sens_f = np.array(sens_f)
spec_f = np.array(spec_f)

with open(fr"./result_train_IRESN_{n_leads}.txt", "w") as output_file:
    print("leads: ", leads, file=output_file)
    print(
        rf"${round(train.mean(), 3)} \pm {round(train.std(), 4)}$ & ${round(val.mean(), 3)} \pm {round(val.std(), 4)}$ & ${round(test.mean(), 3)} \pm {round(test.std(), 4)}$ & ${round(sens.mean(), 3)} \pm {round(sens.std(), 4)}$ & ${round(spec.mean(), 3)} \pm {round(spec.std(), 4)}$", file=output_file
    )
    print(
        rf"${round(train_f.mean(), 3)} \pm {round(train_f.std(), 4)}$ & ${round(val_f.mean(), 3)} \pm {round(val_f.std(), 4)}$ & ${round(test_f.mean(), 3)} \pm {round(test_f.std(), 4)}$ & ${round(sens_f.mean(), 3)} \pm {round(sens_f.std(), 4)}$ & ${round(spec_f.mean(), 3)} \pm {round(spec_f.std(), 4)}$", file=output_file
    )
    print("learned hp", file=output_file)
    for hp in hps:
        print(hp, file=output_file)

    print("training iter", train, file=output_file)
    print("validation iter", val, file=output_file)
    print("test iter", test, file=output_file)
    print("sensitivity iter", sens, file=output_file)
    print("specificity iter", spec, file=output_file)
    print("training direct", train_f, file=output_file)
    print("validation direct", val_f, file=output_file)
    print("test direct", test_f, file=output_file)
    print("sensitivity direct", sens_f, file=output_file)
    print("specificity direct", spec_f, file=output_file)

    w = [[] for _ in range(n_leads)]
    for mod in results:
        coeff = mod["weights"].ravel()[:-1]
        coeff = coeff / np.linalg.norm(coeff)
        splitted = np.split(coeff, n_leads)
        for j, s in enumerate(splitted):
            w[j].append(np.linalg.norm(s) ** 2)
    for i in range(n_leads):
        w[i] = np.array(w[i])
        w[i] = (w[i].mean(), w[i].std())
        print(rf"leads {i}: ${round(w[i][0], 3)} \pm {round(w[i][1], 4)}$", file=output_file)
